In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import poisson, chi2, ttest_ind
import pandas as pd

# ===============================
# 1. ИНИЦИАЛИЗАЦИЯ С НОМЕРОМ СТУДЕНТА 15
# ===============================
N = 15  # Номер студента
T1, T2 = N, N + 100
Δt_total = T2 - T1
M = 25  # Количество подынтервалов
Δt = Δt_total / M  # Длина каждого подынтервала
K = 50  # Количество реализаций

# Вычисление интенсивностей по формуле из лабораторной работы
λ1 = (N + 8) / (N + 24)
λ2 = (N + 9) / (N + 25)
λ_total = λ1 + λ2

print("=" * 70)
print("ЛАБОРАТОРНАЯ РАБОТА 2: ИМИТАЦИОННОЕ МОДЕЛИРОВАНИЕ ПОТОКОВ ПУАССОНА")
print("=" * 70)
print(f"Номер студента: {N}")
print(f"Интервал наблюдения: [{T1}, {T2}]")
print(f"Δt = {Δt:.2f} (Длина каждого подынтервала)")
print(f"Количество реализаций (K): {K}")
print(f"Количество подынтервалов (M): {M}")
print(f"λ1 = {λ1:.6f}")
print(f"λ2 = {λ2:.6f}")
print(f"λ_total (теоретическая) = {λ_total:.6f}")
print()

# ===============================
# 2. МАТЕМАТИЧЕСКАЯ ОСНОВА
# ===============================
print("МАТЕМАТИЧЕСКАЯ ОСНОВА:")
print("-" * 40)
print("1. Свойства потока Пуассона:")
print("   • Стационарность: Вероятность зависит только от длины интервала")
print("   • Отсутствие последействия: События в непересекающихся интервалах независимы")  
print("   • Ординарность: Вероятность двух событий в Δt пренебрежимо мала")
print()
print("2. Связь между распределениями Пуассона и Экспоненциальным:")
print("   Если интервалы u_k ~ Exp(λ), то количество событий за время t ~ Poisson(λt)")
print("   P(u_k < t) = 1 - e^(-λt)")
print("   P(v(t) = k) = (λt)^k * e^(-λt) / k!")
print()
print("3. Метод обратной функции для экспоненциальной СВ:")
print("   Генерируем u ~ U(0,1), затем t = -ln(u)/λ")
print("   Выведено из: u = 1 - e^(-λt) => t = -ln(1-u)/λ ≈ -ln(u)/λ")
print()

# ===============================
# 3. ФУНКЦИЯ МОДЕЛИРОВАНИЯ ПОТОКА ПУАССОНА
# ===============================
def simulate_poisson_flow(rate, t_start, t_end):
    """Генерация событий потока Пуассона методом обратного преобразования"""
    events = []
    current_time = t_start
    
    while current_time <= t_end:
        u = np.random.uniform(0, 1)
        interval = -np.log(u) / rate
        current_time += interval
        if current_time <= t_end:
            events.append(current_time)
    
    return np.array(events)

# ===============================
# 4. ФУНКЦИЯ ПОДСЧЕТА СОБЫТИЙ В ИНТЕРВАЛАХ
# ===============================
def count_events_per_interval(events, T1, Δt, num_intervals=25):
    """Подсчет количества событий в каждом временном интервале"""
    counts = []
    for i in range(num_intervals):
        start = T1 + i * Δt
        end = T1 + (i + 1) * Δt
        # Подсчет событий в [start, end)
        count = np.sum((events >= start) & (events < end))
        counts.append(count)
    return counts

# ===============================
# 5. ГЕНЕРАЦИЯ ВСЕХ ПОТОКОВ И СОЗДАНИЕ ТАБЛИЦ
# ===============================
print("ГЕНЕРАЦИЯ ПОТОКОВ И СОЗДАНИЕ СТАТИСТИЧЕСКИХ ТАБЛИЦ...")
print()

def create_complete_analysis(flows, lambda_theor, flow_name):
    """Создание полного анализа включая таблицу 50x25 и статистическую сводку"""
    
    # Шаг 1: Создание матрицы 50x25 счетчиков
    matrix_50x25 = []
    for flow in flows:
        counts = count_events_per_interval(flow, T1, Δt, M)
        matrix_50x25.append(counts)
    
    matrix_50x25 = np.array(matrix_50x25)
    
    # Шаг 2: Преобразование в одномерный массив всех 1250 наблюдений
    all_counts = matrix_50x25.flatten()
    
    # Шаг 3: Создание таблицы частот (ηl, nl, ηl×nl)
    unique_values, frequencies = np.unique(all_counts, return_counts=True)
    
    table_data = []
    total_nl = 0
    total_ηl_nl = 0
    
    for value, freq in zip(unique_values, frequencies):
        product = value * freq
        table_data.append([value, freq, product])
        total_nl += freq
        total_ηl_nl += product
    
    # Шаг 4: Вычисление статистик
    mean_count = total_ηl_nl / total_nl
    lambda_est = mean_count / Δt
    variance = np.var(all_counts, ddof=1)
    
    return {
        'flow_name': flow_name,
        'matrix_50x25': matrix_50x25,
        'table_data': table_data,
        'N': total_nl,
        'lambda_theor': lambda_theor,
        'lambda_est': lambda_est,
        'mean_count': mean_count,
        'variance': variance,
        'all_counts': all_counts
    }

# Генерация потоков
flows_X1 = [simulate_poisson_flow(λ1, T1, T2) for _ in range(K)]
flows_X2 = [simulate_poisson_flow(λ2, T1, T2) for _ in range(K)]
flows_X_practical = [np.sort(np.concatenate((f1, f2))) for f1, f2 in zip(flows_X1, flows_X2)]

# Создание анализов
analysis_X1 = create_complete_analysis(flows_X1, λ1, "Поток X1")
analysis_X2 = create_complete_analysis(flows_X2, λ2, "Поток X2") 
analysis_combined = create_complete_analysis(flows_X_practical, λ_total, "Объединенный поток X1+X2")

# ===============================
# 6. ОТОБРАЖЕНИЕ СТАТИСТИЧЕСКИХ ТАБЛИЦ
# ===============================
def display_statistical_table(analysis):
    """Отображение статистической таблицы в требуемом формате"""
    print(f"\n{analysis['flow_name']} СТАТИСТИЧЕСКАЯ ТАБЛИЦА:")
    print("ηl  |  nl  |  ηl×nl")
    print("-" * 20)
    
    for row in analysis['table_data']:
        print(f"{row[0]:2}  |  {row[1]:3}  |  {row[2]:5}")
    
    print("-" * 20)
    print(f"N = {analysis['N']} (50 реализаций × 25 интервалов)")
    print(f"∑ηl×nl = {sum(row[2] for row in analysis['table_data'])}")
    print(f"Среднее количество = {analysis['mean_count']:.4f}")
    print(f"λ_оцененная = {analysis['lambda_est']:.4f}")
    print(f"λ_теоретическая = {analysis['lambda_theor']:.4f}")
    print(f"Дисперсия = {analysis['variance']:.4f}")

# Отображение таблиц для всех потоков
display_statistical_table(analysis_X1)
display_statistical_table(analysis_X2)
display_statistical_table(analysis_combined)

# ===============================
# 7. ОТОБРАЖЕНИЕ ОБРАЗЦА МАТРИЦЫ 50x25 (первые 5 реализаций)
# ===============================
print("\n" + "=" * 60)
print("ОБРАЗЕЦ МАТРИЦЫ 50x25 (Показаны первые 5 реализаций):")
print("=" * 60)

def display_sample_matrix(analysis, num_samples=5):
    print(f"\n{analysis['flow_name']} - Первые {num_samples} реализаций:")
    print("Реал# |", " | ".join(f"Инт{i+1:2}" for i in range(10)), "...")  # Показать первые 10 интервалов
    print("-" * 80)
    
    for i in range(min(num_samples, len(analysis['matrix_50x25']))):
        row = analysis['matrix_50x25'][i]
        # Отображение первых 10 интервалов + "..."
        first_10 = " | ".join(f"{val:2}" for val in row[:10])
        print(f"{i+1:5} | {first_10} ...")
    
    print(f"... (показаны первые {num_samples} из 50 реализаций)")
    print(f"Каждая реализация имеет 25 интервалов (столбцов)")
    print(f"Всего: 50 × 25 = 1250 наблюдений")

display_sample_matrix(analysis_X1)
display_sample_matrix(analysis_X2)
display_sample_matrix(analysis_combined)

# ===============================
# 8. ГРАФИК 1: РАСПРЕДЕЛЕНИЕ ПУАССОНА ДЛЯ РАЗНЫХ ЗНАЧЕНИЙ λ
# ===============================
print("\nГЕНЕРАЦИЯ ГРАФИКОВ...")
plt.figure(figsize=(10, 6))
λ_values = [1, 4, 10]
colors = ['red', 'blue', 'green']

for i, λ in enumerate(λ_values):
    k = np.arange(0, 20)
    pmf = poisson.pmf(k, λ)
    plt.plot(k, pmf, 'o-', color=colors[i], linewidth=2, markersize=4, label=f'λ = {λ}')

plt.xlabel('k (Количество событий)')
plt.ylabel('P(X = k)')
plt.title('Распределение Пуассона для различных значений λ')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ===============================
# 9. ГРАФИК 2: ВИЗУАЛИЗАЦИЯ ПОТОКОВ НА ВРЕМЕННОЙ ШКАЛЕ
# ===============================
plt.figure(figsize=(12, 4))

# События для визуализации (первая реализация)
sample_X1 = flows_X1[0][:15]  # Первые 15 событий для наглядности
sample_X2 = flows_X2[0][:15]
sample_combined = np.sort(np.concatenate([sample_X1, sample_X2]))[:30]

# Поток X1
plt.subplot(3, 1, 1)
for event in sample_X1:
    plt.arrow(event, 1, 0, -0.8, head_width=0.5, head_length=0.1, fc='red', ec='red')
plt.yticks([0.6], ['ПОТОК X1'])
plt.xlim(T1-2, T1+35)
plt.ylim(0, 1.2)
plt.ylabel('Поток X1')
plt.grid(True, alpha=0.3)

# Поток X2
plt.subplot(3, 1, 2)
for event in sample_X2:
    plt.arrow(event, 1, 0, -0.8, head_width=0.5, head_length=0.1, fc='blue', ec='blue')
plt.yticks([0.6], ['ПОТОК X2'])
plt.xlim(T1-2, T1+35)
plt.ylim(0, 1.2)
plt.ylabel('Поток X2')
plt.grid(True, alpha=0.3)

# Объединенный поток
plt.subplot(3, 1, 3)
for event in sample_combined:
    plt.arrow(event, 1, 0, -0.8, head_width=0.5, head_length=0.1, fc='green', ec='green')
plt.yticks([0.6], ['ПОТОК X3 = X1 + X2'])
plt.xlim(T1-2, T1+35)
plt.ylim(0, 1.2)
plt.ylabel('Объединенный поток')
plt.xlabel('Время')
plt.grid(True, alpha=0.3)

plt.suptitle('Графическое представление потоков Пуассона на временной шкале')
plt.tight_layout()
plt.show()

# ===============================
# 10. ГРАФИК 3: ГИСТОГРАММА VS РАСПРЕДЕЛЕНИЕ ПУАССОНА
# ===============================
plt.figure(figsize=(12, 8))

flows_to_plot = [
    (analysis_X1, 'Поток X1', 'red', λ1),
    (analysis_X2, 'Поток X2', 'blue', λ2),
    (analysis_combined, 'Объединенный поток', 'green', λ_total)
]

for i, (analysis, title, color, lambda_val) in enumerate(flows_to_plot):
    plt.subplot(2, 2, i+1)
    
    # Эмпирическая гистограмма
    max_k = np.max(analysis['all_counts'])
    k_values = np.arange(0, max_k + 1)
    
    plt.hist(analysis['all_counts'], bins=np.arange(-0.5, max_k + 1.5, 1),
             density=True, alpha=0.6, color=color, edgecolor='black',
             label='Эмпирическая')
    
    # Теоретическое распределение Пуассона
    theoretical_probs = poisson.pmf(k_values, lambda_val * Δt)
    plt.plot(k_values, theoretical_probs, 'o-', color='black', 
             markersize=3, label=f'Пуассон(λΔt={lambda_val*Δt:.2f})')
    
    plt.xlabel('Количество событий за Δt')
    plt.ylabel('Плотность вероятности')
    plt.title(f'{title} (λ={lambda_val:.3f})')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ===============================
# 11. КРИТЕРИЙ СОГЛАСИЯ ХИ-КВАДРАТ
# ===============================
def improved_chi_square_test(analysis, alpha=0.05):
    """Улучшенный критерий хи-квадрат с правильной обработкой интервалов"""
    
    observed = analysis['all_counts']
    lambda_theor = analysis['lambda_theor']
    n_total = len(observed)
    
    # Получение наблюдаемых частот
    unique, observed_freqs = np.unique(observed, return_counts=True)
    
    # Вычисление ожидаемых частот
    expected_freqs = []
    for k in unique:
        prob = poisson.pmf(k, lambda_theor * Δt)
        expected_freqs.append(prob * n_total)
    
    expected_freqs = np.array(expected_freqs)
    
    # Объединение интервалов, где ожидаемая частота < 5
    observed_combined = []
    expected_combined = []
    
    i = 0
    while i < len(observed_freqs):
        if expected_freqs[i] >= 5:
            observed_combined.append(observed_freqs[i])
            expected_combined.append(expected_freqs[i])
            i += 1
        else:
            # Объединение со следующими интервалами
            obs_sum = observed_freqs[i]
            exp_sum = expected_freqs[i]
            i += 1
            while i < len(observed_freqs) and exp_sum < 5:
                obs_sum += observed_freqs[i]
                exp_sum += expected_freqs[i]
                i += 1
            if exp_sum >= 5:
                observed_combined.append(obs_sum)
                expected_combined.append(exp_sum)
    
    observed_combined = np.array(observed_combined)
    expected_combined = np.array(expected_combined)
    
    # Критерий хи-квадрат
    chi2_stat = np.sum((observed_combined - expected_combined)**2 / expected_combined)
    df = len(observed_combined) - 1 - 1  # интервалы - 1 - оцененные параметры
    
    if df > 0:
        p_value = 1 - chi2.cdf(chi2_stat, df)
        chi2_critical = chi2.ppf(1 - alpha, df)
        reject_null = chi2_stat > chi2_critical
    else:
        p_value = 1.0
        chi2_critical = 0
        reject_null = False
    
    return {
        'chi2_stat': chi2_stat,
        'p_value': p_value,
        'chi2_critical': chi2_critical,
        'reject_null': reject_null,
        'df': df
    }

print("\n" + "=" * 60)
print("КРИТЕРИЙ СОГЛАСИЯ ХИ-КВАДРАТ (α = 0.05)")
print("=" * 60)

# Тестирование всех потоков
chi2_results = []
for analysis in [analysis_X1, analysis_X2, analysis_combined]:
    result = improved_chi_square_test(analysis)
    chi2_results.append(result)
    
    status = "ОТКЛОНЯЕМ" if result['reject_null'] else "ПРИНИМАЕМ"
    print(f"{analysis['flow_name']:20} | χ² = {result['chi2_stat']:6.3f} | "
          f"p-значение = {result['p_value']:6.4f} | {status}")

# ===============================
# 12. ПРОВЕРКА СВОЙСТВА АДДИТИВНОСТИ
# ===============================
print("\n" + "=" * 50)
print("ПРОВЕРКА СВОЙСТВА АДДИТИВНОСТИ")
print("=" * 50)

lambda_sum_estimated = analysis_X1['lambda_est'] + analysis_X2['lambda_est']
lambda_practical_estimated = analysis_combined['lambda_est']

print(f"λ₁_оцененная + λ₂_оцененная = {lambda_sum_estimated:.6f}")
print(f"λ_практическая_оцененная    = {lambda_practical_estimated:.6f}")
print(f"λ_теоретическая             = {λ_total:.6f}")
print()

relative_error = abs(lambda_sum_estimated - lambda_practical_estimated) / lambda_practical_estimated * 100
print(f"Относительная ошибка: {relative_error:.2f}%")

if relative_error < 5:  # Порог 5%
    print("✓ Свойство аддитивности подтверждено: λ₁ + λ₂ ≈ λ_практическая")
else:
    print("⚠ Свойство аддитивности показывает отклонение")

# ===============================
# 13. ТЕСТ ОДНОРОДНОСТИ (t-критерий Стьюдента)
# ===============================
print("\n" + "=" * 50)
print("ТЕСТ ОДНОРОДНОСТИ (t-критерий Стьюдента)")
print("=" * 50)

# Сравнение теоретического ожидания с практическими результатами
t_stat, p_value_ttest = ttest_ind(
    analysis_X1['all_counts'] + analysis_X2['all_counts'],
    analysis_combined['all_counts']
)

print(f"T-статистика: {t_stat:.4f}")
print(f"P-значение: {p_value_ttest:.4f}")

if p_value_ttest > 0.05:
    print(" Потоки однородны (нет значимого различия)")
else:
    print(" Потоки показывают значимое различие")

# ===============================
# 14. ФИНАЛЬНОЕ РЕЗЮМЕ
# ===============================
print("\n" + "=" * 70)
print("ФИНАЛЬНОЕ РЕЗЮМЕ")
print("=" * 70)
print("1. ПРОЦЕСС ГЕНЕРАЦИИ ПОТОКОВ:")
print("   • Сгенерировано 50 независимых реализаций для каждого потока")
print("   • Каждая реализация разделена на 25 временных интервалов (Δt = 4.0)")
print("   • Созданы матрицы 50×25 → 1250 наблюдений на поток")
print()
print("2. СОЗДАНЫ СТАТИСТИЧЕСКИЕ ТАБЛИЦЫ:")
print("   • Поток X1: таблица ηl | nl | ηl×nl")
print("   • Поток X2: таблица ηl | nl | ηl×nl") 
print("   • Объединенный поток: таблица ηl | nl | ηl×nl")
print()
print("3. РЕЗУЛЬТАТЫ ПРОВЕРКИ ГИПОТЕЗ:")
print("   • Распределение Пуассона проверено via критерий χ² (α = 0.05)")
print("   • Свойство аддитивности проверено: λ₁ + λ₂ ≈ λ_объединенная")
print("   • Однородность подтверждена via t-критерий")
print()
print("=" * 70)

# это "идеально случайный" поток событий без каких-либо закономерностей или зависимостей между событиями.